In [18]:
import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU
import glob
import matplotlib.pyplot as plt
from math import floor

+++ Start For Google Colab



from google.colab import drive
drive.mount('/content/drive')

# Grab all the bounding box info
with open('/content/drive/My Drive/boundingBoxes.txt', 'r') as f:
    lines = f.readlines()
 
boxes = [[float(x) for x in line.strip().split(",")] for line in lines]

# Import image paths
imagesRaw = glob.glob("/content/drive/My Drive/new_images/*.jpg")
imagesRaw.sort()

for e in range(len(imagesRaw)):
  loc = imagesRaw[e].find(" (1)")
  if loc != -1:
    imagesRaw[e] = imagesRaw[e][:loc] + imagesRaw[e][loc + 4:]

+++ End For Google Colab

In [19]:
# Check if install with cuda and gpu avaiable:
print(tf.test.is_built_with_cuda)
print(tf.config.list_physical_devices('GPU') )

<function is_built_with_cuda at 0x000001A08D121C10>
[]


In [20]:
# Define the constants
S = 7 # Divide the image to have S*S cells
C = 1 # Number of classes 
B = 2 # Number of bounding boxes to be predicted per cell
BATCH_SIZE = 1

In [21]:
# Import image paths
imagesRaw = glob.glob("./new_images/*.jpg")
imagesRaw.sort()

In [22]:
# Grab all the bounding box info
with open('boundingBoxes.txt', 'r') as f:
    lines = f.readlines()
 
boxes = [[float(x) for x in line.strip().split(",")] for line in lines]

In [23]:
def load_jpeg(image, box):
    # Load the image turn into jpeg
    decoded = tf.io.read_file(image)
    imageTf = tf.image.decode_jpeg(decoded)
    # Normalize the images
    returnImage = tf.cast(imageTf, tf.float32) / 255.0
    returnImage = tf.reshape(returnImage, (488,488,3))
    
    return returnImage, box

def convertToYTrue(box):
    # This inserts the BOX to correct GRID POSITION
    row, col = box[1], box[2]
    updates = tf.constant([box[3:]])
    # Could simplify this using tf.meshgrid somehow
    idx = tf.constant([[[row, col, 0], [row, col, 1], [row, col, 2], [row, col, 3], [row, col, 4], [row, col, 5]]])
    output = tf.ones([7,7,6]) + [-1.0, -0.1, -.1, -.1, -.1, -1.0]
    output = tf.tensor_scatter_nd_add(output, idx, tf.zeros_like(updates) + [0, -0.9, -0.9, -0.9, -0.9, 0])
    output = tf.tensor_scatter_nd_add(output, idx, updates)
    return output
    
# Plots two given images
def display(imageOne, imageTwo):
    plt.figure(figsize=(15, 15))
    plt.subplot(1,2,1)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(imageOne))
    plt.axis("off")
    plt.subplot(1,2,2)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(imageTwo))
    plt.axis("off")
    plt.show()

#Convert true pixel indexes to yolo format
def convertToYoloFormat(pixelPositions, num_boxes=1, imageWidth=488, imageHeight=488):
    # Format is [containsObj, center_x, center_y, width, height, classes...]
    cellHeight = imageHeight / S
    cellWidth = imageWidth / S
    returnBox = []
    countStart = 0
    countEnd = 1
    storage = []
    while countStart < num_boxes:
        _ ,xMin, xMax, yMin, yMax = pixelPositions[5*countStart:4*countEnd + 1]
        # Calculate the exact pixel index of the center of the box w.r.t the image
        pointX = (floor((xMax - xMin)/2)) + xMin
        pointY = (floor((yMax - yMin)/2)) + yMin
        # Calculate the center of the box w.r.t the cell
        centerX = (pointX - floor(pointX/cellWidth) * cellWidth)/cellWidth
        centerY = (pointY - floor(pointY/cellHeight) * cellHeight)/cellHeight
        # Calculate the dimensions w.r.t the cell
        width = (xMax - xMin)/imageWidth
        height = (yMax - yMin)/imageHeight
        
        returnBox = returnBox + [_, centerX, centerY, width, height]
        countEnd += 1
        countStart += 1
        
    colIndex = floor(pointX/cellWidth)
    rowIndex = floor(pointY/cellHeight)
    storage += colIndex, rowIndex
    
    return [num_boxes] + storage + returnBox + pixelPositions[5*countStart:]

def generateIndexes():
    table = []
    for i in range(BATCH_SIZE):
        for j in range(S*S):
            for k in range(B):
                table.append([i,j,k])
    return table

# Convert the yolo box format to tensor box format NEED TO UPDATE ++++
def convertFromYolo(box, image, index):
    xMax = box[0]*488*2
    yMax = box[1]*488*2
    xMin = - box[2]*488 + xMax
    yMin = - box[3]*488 + yMax
    return xMax, yMax, xMin, yMin
    

In [24]:
convertToYoloFormat(boxes[0])

[1,
 3,
 3,
 1.0,
 0.24180327868852508,
 0.1844262295081972,
 0.8360655737704918,
 0.5840163934426229,
 1.0]

In [25]:
# Convert boxes to yolo format:
convertedBoxes = [convertToYoloFormat(box) for box in boxes]
# Convert the convertedBoxes to the same format as y_pred
yTrue = [convertToYTrue(box) for box in convertedBoxes]
# Create a new dataset, pairing images and respective boxes
imageDataSet = tf.data.Dataset.from_tensor_slices((imagesRaw, yTrue))
# Shuffle the data
imageDataSet = imageDataSet.shuffle(1000)
# Map the paths to turn into images
entireSet = imageDataSet.map(load_jpeg)

In [36]:
for image, box in entireSet.take(1):
    for i in range(image.shape[0]):
        print(image[i])

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1

 [0.69803923 0.6313726  0.5529412 ]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[0.6784314  0.627451   0.5647059 ]
 [0.6784314  0.627451   0.56078434]
 [0.67058825 0.61960787 0.5568628 ]
 ...
 [0.7019608  0.63529414 0.5568628 ]
 [0.69803923 0.6313726  0.5529412 ]
 [0.69411767 0.63529414 0.5529412 ]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[0.6784314  0.62352943 0.57254905]
 [0.6745098  0.62352943 0.56078434]
 [0.67058825 0.6156863  0.5647059 ]
 ...
 [0.7019608  0.6431373  0.56078434]
 [0.7019608  0.6431373  0.56078434]
 [0.7019608  0.6431373  0.56078434]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[0.67058825 0.6156863  0.57254905]
 [0.67058825 0.6156863  0.5647059 ]
 [0.67058825 0.6156863  0.57254905]
 ...
 [0.70980394 0.6509804  0.56078434]
 [0.70980394 0.6509804  0.56078434]
 [0.7019608  0.654902   0.56078434]], shape=(488, 3), dtype=float32)
tf.Tensor(
[[0.67058825 0.6156863  0.57254905]
 [0.67058825 0.6156863  0.57254905]
 [0.67058825 0.6156863  0.57254905]
 ...
 [0.7137255 

In [26]:
# Divide the dataset into train, test and validation
trainSize = int(0.8 * 2594)
valSize = int(0.1 * 2594)
train = entireSet.take(trainSize)
temp = entireSet.skip(trainSize)
test = temp.skip(valSize)
validation = temp.take(valSize)

In [27]:
# Create batches:
train_batches = train.batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
#test_batches = test.batch(BATCH_SIZE)

In [28]:
model = tf.keras.Sequential([
    #First Layer
    tf.keras.layers.Conv2D(64, (7,7), strides=(2, 2),  input_shape=(488,488,3)),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    
    #Second Layer
    tf.keras.layers.Conv2D(192, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    
    #Third Layer    
    tf.keras.layers.Conv2D(128, (1,1),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(256, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(256, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(512, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    
    #Fourth Layer
    # +++ Repeated block
    tf.keras.layers.Conv2D(256, (1,1),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(512, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(256, (1,1),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(512, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(256, (1,1),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(512, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(256, (1,1),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(512, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    # +++ END BLOCK
    tf.keras.layers.Conv2D(512, (1,1),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(1024, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2,2)),
    
    #Fifth layer
    # +++ Repeated Block
    tf.keras.layers.Conv2D(512, (1,1),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(1024, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(512, (1,1),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    tf.keras.layers.Conv2D(1024, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    # +++ END BLOCK
    tf.keras.layers.Conv2D(1024, (3,3),  strides=(2, 2), padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(1024, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    #Sixth Layer
    tf.keras.layers.Conv2D(1024, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),

    tf.keras.layers.Conv2D(1024, (3,3),  padding="same"),
    tf.keras.layers.BatchNormalization(),
    LeakyReLU(alpha=0.3),
    

    # Final Output Layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096),
    tf.keras.layers.Dense(S * S * (B*5+C), input_shape=(4096,), activation="sigmoid"),
    tf.keras.layers.Reshape(target_shape = (S, S, (B*5+C)))
    
])

The yoloLoss function is an adaptation from here: https://blog.emmanuelcaradec.com/humble-yolo-implementation-in-keras/. Credit to Emmanuel Caradec. 

A helpful overview of the equation:
![](https://miro.medium.com/max/1400/1*1noFJp_rnXsG0TJvtoghFA.png)

In [29]:
def yoloLoss(y_true, y_pred):
    # Define constants:
    lambdaCoord = 5
    lambdaNoob = 0.5

    #grid = [[[float(x),float(y)]]*B for y in range(S) for x in range(S)]

    true_boxes = tf.reshape(y_true[...,:5], [-1,S*S,1,5])
    pred_boxes = tf.reshape(y_pred[...,:B*5], (-1,S*S,B,5))
    # Reshape the 3: too S*S, B, 5 i.e. 49, 2, 5. 
    # Each cell has two bounding boxes, with each bounding box, having 5 elements. 
    
    # Calculate the IOU for every box.
    num = tf.math.multiply(true_boxes[...,1:5], pred_boxes[...,1:5])
    denom = true_boxes[...,1:5] + pred_boxes[...,1:5] - num
    iou = tf.math.reduce_sum(num/denom,  axis=-1)

    # If boxes in the same cell have the same IOU, add 1 to the first box (since they are both equal) 
    duplicates = tf.where(tf.equal(iou[...,0], iou[...,1]))
    iou = tf.tensor_scatter_nd_add(iou, duplicates, tf.add(tf.zeros_like(duplicates, dtype=tf.float32), [1,0]))
    
    # Get the indices of the max IOUs for each cell and grab the respetive boxes.
    maxIou = tf.math.reduce_max(iou, axis=-1, keepdims=True)
    maxIdx = tf.where(tf.equal(iou, maxIou))
    # Use the best boxes for calculations in the xy_loss, wh_loss, and conf_loss (as per the formula)
    best_boxes = tf.reshape(tf.gather_nd(pred_boxes, maxIdx), [-1, S*S,1,5])

    # The confidence on whether there is an object in the cell, between 0-1
    y_pred_conf = best_boxes[...,0]
    y_true_conf = true_boxes[...,0]
    y_true_conf_noob = tf.math.subtract(tf.ones_like(y_true_conf), y_true_conf)
    
    # The width and height of the bounding boxes, normalised to the image size
    y_pred_wh   = best_boxes[...,3:5]
    y_true_wh   = true_boxes[...,3:5]

    # The centre of the bounding box, normalised to the cell size
    y_pred_xy   = best_boxes[...,1:3]
    y_true_xy   = true_boxes[...,1:3]
    
    # The classes that are within this cell. 0 if  no, 1 if yes. Noting that there is only one class in this instance. 
    y_true_class = tf.reshape(y_true[...,5:], [-1, S*S, C]) 
    y_pred_class = tf.reshape(y_pred[...,B*5:], [-1, S*S, C])
    
    # Losses Calculations +++++++++++++++++++++++++++++++++++++++++++++++++++
    #y_true_conf will 0 cells that do not have an object in the ground truth 
    xy_loss = lambdaCoord * tf.math.reduce_sum(tf.math.reduce_sum(tf.math.square(y_true_xy - y_pred_xy),
                                                       axis=-1)*y_true_conf, axis=-1)
  
    
    # Two reduce sums  = 2 summations. Axis = -1, along last dimensions i.e. columns/entries in this case.
    # Square is element wise. y_true_conf will 0 cells that do not have an object in the ground truth 
    wh_loss = lambdaCoord * tf.math.reduce_sum(tf.math.reduce_sum(tf.math.square(tf.math.sqrt(y_true_wh) - tf.math.sqrt(y_pred_wh)), axis=-1)*y_true_conf, axis=-1)      

    #y_true_conf will 0 cells that do not have an object in the ground truth 
    #y_true_conf_noob will 0 cells that DO have an object in the ground truth 
    conf_loss = tf.math.reduce_sum(tf.math.square(y_true_conf - y_pred_conf)*y_true_conf, axis=-1)
    conf_loss_noob = lambdaNoob * tf.math.reduce_sum(tf.math.square(y_true_conf - y_pred_conf)*y_true_conf_noob, axis=-1)

    clss_loss  = tf.math.reduce_sum(tf.math.square(y_true_class - y_pred_class)*y_true_conf, axis=-1) 

    loss =  clss_loss + xy_loss + wh_loss + conf_loss + conf_loss_noob
    
    #print("\n","*"*10, "Y_PRED: \n", y_pred)
    #print("\n","*"*10, "Y_TRUE: \n", y_true)
    #print("\n","*"*10, "XY LOSS: \n", xy_loss)
    #print("\n","*"*10, "WH LOSS: \n", wh_loss)
    #print("\n","*"*10,"conf_loss: \n", conf_loss)
    #print("\n","*"*10, "conf_noob: \n",y_true_conf_noob)
    #print("\n","*"*10, "conf_loss noob: \n",conf_loss_noob)
    #print("\n","*"*10, "CLASS LOSS: \n",  clss_loss)
    #print("\n","*"*10, loss)

    return loss

In [30]:
def jaccardIndex(y_true, y_pred):
    """
    Compares y_true with highest prob y_pred box
    """
    y_true = tf.reshape(y_true[...,:5], [-1,S*S,1,5])
    y_pred = tf.reshape(y_pred[...,:B*5], [-1,S*S,B,5])
    
    y_true_box = tf.gather_nd(y_true, (tf.where(tf.equal(y_true[...,0], tf.math.reduce_max(y_true[...,0]))))[0])
    y_pred_box = tf.gather_nd(y_pred, (tf.where(tf.equal(y_pred[...,0], tf.math.reduce_max(y_pred[...,0]))))[0])

    numerator = (tf.math.reduce_sum(tf.math.multiply(y_true_box[...,1:5], y_pred_box[...,1:5])))
    # Calculate the area of both - area that overlaps
    denom = tf.math.reduce_sum(y_true_box[...,1:5]) + tf.math.reduce_sum(y_pred_box[...,1:5]) - numerator
    iou = numerator/denom
    
    return iou

In [31]:
# This max supression will only work when they're only exists a single class. Can be adapted.  
def nonMaxSupression(y_true, y_pred):
    maxTrueValue = tf.gather_nd(y_true, (tf.where(tf.equal(y_true[...,4], tf.math.reduce_max(y_true[...,4]))))[0])
    # Grabs the max value along index 4 (reduce_max), then finds the index of the value in the tensor
    maxPredIndex = (tf.where(tf.equal(y_pred[...,4], tf.math.reduce_max(y_pred[...,4]))))[0]
    # Grabs the actual value, using index calculated above
    maxPredValue = tf.gather_nd(y_pred, maxPredIndex)
    idx = generateIndexes()
    # Disgusting time complexity :D
    for i in range(len(idx)):
        box = tf.gather_nd(y_pred, idx[i])
        val = iou(maxPredValue, box)
        if val < 0.5:
            del idx[i]

In [32]:
y_pred = tf.zeros([BATCH_SIZE, 7, 7, 11])
for image, box in train_batches.take(1):
    y_true = box
    #try:
      #output = yoloLoss(y_true, y_pred)
    #except:
      #print("*" * 10,y_true)
      #print('\n', "*" * 10, y_pred) 
#jaccard = jaccardIndex(y_true, y_pred)

In [33]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.),
              loss=yoloLoss,
              metrics=[jaccardIndex], run_eagerly=True)

In [34]:
#model.summary()

In [35]:
history =  model.fit(train_batches, epochs=200, validation_data=validation_batches)

Epoch 1/200
   7/2075 [..............................] - ETA: 2:00:55 - loss: 0.1750 - jaccardIndex: 0.3401

InvalidArgumentError: in user code:

    C:\Users\e_gui\AppData\Local\Temp/ipykernel_15068/2658823159.py:9 jaccardIndex  *
        y_pred_box = tf.gather_nd(y_pred, (tf.where(tf.equal(y_pred[...,0], tf.math.reduce_max(y_pred[...,0]))))[0])
    C:\Users\e_gui\anaconda3\envs\tfEnv\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
        return target(*args, **kwargs)
    C:\Users\e_gui\anaconda3\envs\tfEnv\lib\site-packages\tensorflow\python\ops\array_ops.py:1041 _slice_helper
        return strided_slice(
    C:\Users\e_gui\anaconda3\envs\tfEnv\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
        return target(*args, **kwargs)
    C:\Users\e_gui\anaconda3\envs\tfEnv\lib\site-packages\tensorflow\python\ops\array_ops.py:1214 strided_slice
        op = gen_array_ops.strided_slice(
    C:\Users\e_gui\anaconda3\envs\tfEnv\lib\site-packages\tensorflow\python\ops\gen_array_ops.py:10510 strided_slice
        _ops.raise_from_not_ok_status(e, name)
    C:\Users

In [ ]:
for image, box in test.take(1):
  y_true = box
  predictImage = tf.expand_dims(image, axis=0)
  result = model.predict(predictImage)

In [ ]:
y_true = tf.reshape(y_true[...,:5], [-1,S*S,1,5])
y_pred = tf.reshape(result[...,:B*5], [-1,S*S,B,5])

y_true_box = tf.gather_nd(y_true, (tf.where(tf.equal(y_true[...,0], tf.math.reduce_max(y_true[...,0]))))[0])
y_pred_box = tf.gather_nd(y_pred, (tf.where(tf.equal(y_pred[...,0], tf.math.reduce_max(y_pred[...,0]))))[0])

In [ ]:
print(y_true_box)

In [ ]:
print(y_pred_box)

In [ ]:
print(jaccardIndex(box, result))

In [ ]:
5.5345783 * 1 + 0.44262296 + 1.7428291 + 